In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import os 
import seaborn as sns
from sklearn.model_selection import train_test_split
import pickle
import tensorflow as tf
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler,  RobustScaler
import pickle
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout
from keras import regularizers, layers, optimizers, initializers
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

from tensorflow.keras.applications import EfficientNetV2M
import numpy as np
import gc

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Input, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
print(f'Current GPU allocator: {os.getenv("TF_GPU_ALLOCATOR")}')

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            print(f'Setting memory growth for {gpu}')
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')
    

2024-04-23 06:14:00.863709: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-23 06:14:01.423590: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Current GPU allocator: cuda_malloc_async
Setting memory growth for PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3080, compute capability 8.6


2024-04-23 06:14:02.546659: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-23 06:14:02.562148: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-23 06:14:02.562214: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-23 06:14:02.563051: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
study_name = '422_effitesti_ilmanskaleria_3'

In [3]:

mean_columns = ['X4_mean', 'X11_mean', 'X18_mean', 'X50_mean', 'X26_mean', 'X3112_mean']

In [4]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')

In [5]:
# train_df = train_df[:1000]
# test_df = test_df[:1000]

In [6]:
sd_columns = [col for col in train_df.columns if col.endswith('_sd')]
train_df.drop(columns=sd_columns, inplace=True)

In [7]:
train_images_path = './data/train_images/'
test_images_path = './data/test_images/'    

train_df['image_path'] = train_df['id'].apply(lambda x: os.path.join(train_images_path, f'{x}.jpeg'))
test_df['image_path'] = test_df['id'].apply(lambda x: os.path.join(test_images_path, f'{x}.jpeg'))

In [8]:
for column in mean_columns:
    lower_quantile = train_df[column].quantile(0.005)
    upper_quantile = train_df[column].quantile(0.985)  
    train_df = train_df[(train_df[column] >= lower_quantile) & (train_df[column] <= upper_quantile)]

In [9]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for i, trait in enumerate(mean_columns):

    # Determine the bin edges dynamically based on the distribution of traits
    bin_edges = np.percentile(train_df[trait], np.linspace(0, 100, 5 + 1))
    train_df[f"bin_{i}"] = np.digitize(train_df[trait], bin_edges)

# Concatenate the bins into a final bin
train_df["final_bin"] = (
    train_df[[f"bin_{i}" for i in range(len(mean_columns))]]
    .astype(str)
    .agg("".join, axis=1)
)

# Perform the stratified split using final bin
train_df = train_df.reset_index(drop=True)
for fold, (train_idx, valid_idx) in enumerate(skf.split(train_df, train_df["final_bin"])):
    train_df.loc[valid_idx, "fold"] = fold

/home/tobias/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [10]:
train_original = train_df.copy()
train_plot = train_df.copy()
sample_df = train_df.copy()
train_df = sample_df[sample_df.fold != 3]
valid_df = sample_df[sample_df.fold == 3]
print(f"# Num Train: {len(train_df)} | Num Valid: {len(valid_df)}")

# Num Train: 39335 | Num Valid: 9833


In [11]:

from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()

y_train_original = train_df[mean_columns].values
y_valid_original = valid_df[mean_columns].values

train_df[mean_columns] = np.log10(train_df[mean_columns])
valid_df[mean_columns] = np.log10(valid_df[mean_columns])

train_df[mean_columns] = std_scaler.fit_transform(train_df[mean_columns])
valid_df[mean_columns] = std_scaler.transform(valid_df[mean_columns])


def r2_score_tf(y_true, y_pred):

    try: 
        ss_res = tf.reduce_sum(tf.square(y_true - y_pred), axis=0)
        ss_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true, axis=0)), axis=0)
        r2 = 1 - ss_res/(ss_tot + tf.keras.backend.epsilon())
        r2 = tf.where(tf.math.is_nan(r2), tf.zeros_like(r2), r2) 
        return tf.reduce_mean(tf.maximum(r2, 0.0))
    except Exception as e:
        # print(f'Error in r2_score_tf: {e}')
        return float('-inf')
    
custom_objects = {"r2_score_tf": r2_score_tf}




        

/tmp/ipykernel_282380/2903089625.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[mean_columns] = np.log10(train_df[mean_columns])
/tmp/ipykernel_282380/2903089625.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df[mean_columns] = np.log10(valid_df[mean_columns])
/tmp/ipykernel_282380/2903089625.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [28]:
import tensorflow_addons as tfa

def augment_image(img):
  
    img = img / 255.0
    

    img = tf.image.random_flip_left_right(img)
    
    img = tf.image.rot90(img, k=tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))

    img = tf.image.random_brightness(img, max_delta=0.1)
    img = tf.image.random_hue(img, max_delta=0.1)
    img = tf.image.random_saturation(img, lower=0.9, upper=1.1)
    img = tf.image.random_contrast(img, lower=0.9, upper=1.1)

    img = tf.image.random_jpeg_quality(img, min_jpeg_quality=85, max_jpeg_quality=100)

    crop_size = tf.random.uniform(shape=[], minval=420, maxval=480, dtype=tf.int32)
    img = tf.image.random_crop(img, size=[crop_size, crop_size, 3])
    img = tf.image.resize(img, [480, 480]) 


    theta = tf.random.uniform([], minval=-0.05, maxval=0.05)
    tx = tf.random.uniform([], minval=-40, maxval=40)
    ty = tf.random.uniform([], minval=-40, maxval=40)
    # Adding a small rotation component
    cos_theta = tf.cos(theta)
    sin_theta = tf.sin(theta)
    transformation_matrix = [cos_theta, -sin_theta, tx,
                                sin_theta, cos_theta, ty,
                                0, 0]
    img = tfa.image.transform(img, transformation_matrix, interpolation="BILINEAR")

    img = img * 255.0


    return img


def process_image(file_path):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (480, 480))
    img = augment_image(img)  
    # img = tf.cast(img, tf.uint8)    
    return img

def process_image_valid(file_path):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (480, 480))
    # img = tf.cast(img, tf.uint8)
    return img



# Define your dataset processing function
def process_path_train(file_path, targets):
    img = process_image(file_path)
    return img, targets


def process_path_valid(file_path, targets):
    img = process_image_valid(file_path)
    return img, targets

def process_path_test(file_path):
    img = process_image_valid(file_path)
    img = tf.expand_dims(img, 0)
    return img


In [13]:
BUFFER_SIZE = 50000 
EPOCHS = 10
BATCH_SIZE = 10

y_train = train_df[mean_columns].values
y_valid = valid_df[mean_columns].values

train_images_path = train_df['image_path'].values
valid_images_path = valid_df['image_path'].values

train_dataset = tf.data.Dataset.from_tensor_slices((train_images_path, y_train))
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
valid_dataset = tf.data.Dataset.from_tensor_slices((valid_images_path, y_valid))

train_dataset = train_dataset.map(process_path_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.map(process_path_valid, num_parallel_calls=tf.data.experimental.AUTOTUNE)

train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

len_train = len(train_dataset) * EPOCHS

print(f'LR schedule steps: {len_train}')




2024-04-23 06:14:04.173056: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-23 06:14:04.173168: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-23 06:14:04.173218: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-23 06:14:04.299269: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-23 06:14:04.299373: I tensorflow/compile

LR schedule steps: 39340


In [14]:
#####  KAKSIHAARAINEN AVG MAX IMAGELLA #####



import tensorflow as tf
import numpy as np

def r2_score_tf(y_true, y_pred):

    try: 
        ss_res = tf.reduce_sum(tf.square(y_true - y_pred), axis=0)
        ss_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true, axis=0)), axis=0)
        r2 = 1 - ss_res/(ss_tot + tf.keras.backend.epsilon())
        r2 = tf.where(tf.math.is_nan(r2), tf.zeros_like(r2), r2) 
        return tf.reduce_mean(tf.maximum(r2, 0.0))
    except Exception as e:
        # print(f'Error in r2_score_tf: {e}')
        return float('-inf')


# Asetetaan syötteet
image_input_avg = Input(shape=(480, 480, 3), name='image_input_avg')


# Luo perus EfficientNetV2M mallit
eff_avg_base = EfficientNetV2M(weights='imagenet', include_top=False, pooling='avg', input_tensor=image_input_avg)
eff_avg_base.trainable = True

dropout = Dropout(0.5)(eff_avg_base.output)

output = Dense(6, activation='linear')(dropout)

finetune_model = Model(inputs=image_input_avg, outputs=output, name='finetune_model')


lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    1e-4,    
    alpha=0.05,
    name="CosineDecay",
    decay_steps=len_train
)

# Aseta oppimisnopeuden aikataulu
finetune_model.compile(optimizer=optimizers.RMSprop(learning_rate=lr_schedule), loss='mae', metrics=['mse', 'mae', 'mape', r2_score_tf])






In [15]:
trainable_count = sum([tf.size(v).numpy() for v in finetune_model.trainable_weights])
non_trainable_count = sum([tf.size(v).numpy() for v in finetune_model.non_trainable_weights])
print(f"Total parameters: {trainable_count + non_trainable_count:,}")
print(f"Trainable parameters: {trainable_count:,}")
print(f"Non-trainable parameters: {non_trainable_count:,}")


Total parameters: 53,158,074
Trainable parameters: 52,866,042
Non-trainable parameters: 292,032


In [16]:
import tensorflow as tf
from tensorflow.keras.callbacks import Callback
import matplotlib.pyplot as plt
import io

class TrainImageLoggingCallback(Callback):
    def __init__(self, log_dir, data):
        super(TrainImageLoggingCallback, self).__init__()
        self.log_dir = log_dir
        self.data = data
        self.writer = tf.summary.create_file_writer(log_dir)

    def on_epoch_end(self, epoch, logs=None):
        # Fetch a batch of images
        for imgs, _ in self.data.take(1):  # Adjust depending on your dataset structure
                        
            augmented_images = tf.cast(imgs, tf.uint8)    
        
            # Prepare the image to write to TensorBoard
            with self.writer.as_default():
                tf.summary.image("Augmented Images", augmented_images, step=epoch, max_outputs=10)

            self.writer.flush()


In [17]:
log_folder = f"./logs/effi/trial_{study_name}"
print(f'Logging to {log_folder}')
os.makedirs(log_folder, exist_ok=True)

# Aseta logitiedostojen hakemisto
tensorboard_callback = TensorBoard(log_dir=log_folder, histogram_freq=1, update_freq='batch')

Logging to ./logs/effi/trial_422_effitesti_ilmanskaleria_3


In [18]:

tf.keras.backend.clear_session()
gc.collect()

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath=f'./NN_search/testifinetus_{study_name}.h5', monitor='val_mae', save_best_only=True, save_weights_only=True, mode = 'min',  verbose = 1),
    tensorboard_callback,
    TrainImageLoggingCallback(log_folder, train_dataset)     
]

history = finetune_model.fit(train_dataset, validation_data=valid_dataset, epochs=EPOCHS, verbose=1, callbacks=callbacks)




Epoch 1/10


2024-04-23 06:14:09.687398: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [39335]
	 [[{{node Placeholder/_0}}]]
2024-04-23 06:14:09.687667: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [39335]
	 [[{{node Placeholder/_0}}]]
2024-04-23 06:14:53.171260: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600


3934/3934 [==============================] - ETA: 0s - loss: 0.7056 - mse: 0.8157 - mae: 0.7056 - mape: 315.7946 - r2_score_tf: 0.1735

2024-04-23 06:43:57.257784: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [9833,6]
	 [[{{node Placeholder/_1}}]]
2024-04-23 06:43:57.258056: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [9833,6]
	 [[{{node Placeholder/_1}}]]



Epoch 1: val_mae improved from inf to 0.66721, saving model to ./NN_search/testifinetus_422_effitesti_ilmanskaleria_3.h5


...

3934/3934 [==============================] - 1792s 455ms/step - loss: 0.5617 - mse: 0.5565 - mae: 0.5617 - mape: 331.6313 - r2_score_tf: 0.3786 - val_loss: 0.6108 - val_mse: 0.6541 - val_mae: 0.6108 - val_mape: 424.3371 - val_r2_score_tf: 0.3103
Epoch 9/10
3934/3934 [==============================] - ETA: 0s - loss: 0.5504 - mse: 0.5369 - mae: 0.5504 - mape: 325.0998 - r2_score_tf: 0.3973
Epoch 9: val_mae did not improve from 0.61079


2024-04-23 10:47:10.107792: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [39335,6]
	 [[{{node Placeholder/_1}}]]
2024-04-23 10:47:10.108366: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [39335,6]
	 [[{{node Placeholder/_1}}]]


3934/3934 [==============================] - 1800s 458ms/step - loss: 0.5504 - mse: 0.5369 - mae: 0.5504 - mape: 325.0998 - r2_score_tf: 0.3973 - val_loss: 0.6112 - val_mse: 0.6590 - val_mae: 0.6112 - val_mape: 440.6141 - val_r2_score_tf: 0.3102
Epoch 10/10
3934/3934 [==============================] - ETA: 0s - loss: 0.5439 - mse: 0.5271 - mae: 0.5439 - mape: 322.1861 - r2_score_tf: 0.4066
Epoch 10: val_mae did not improve from 0.61079


2024-04-23 11:17:07.291210: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [39335,6]
	 [[{{node Placeholder/_1}}]]
2024-04-23 11:17:07.292091: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [39335]
	 [[{{node Placeholder/_0}}]]


3934/3934 [==============================] - 1797s 457ms/step - loss: 0.5439 - mse: 0.5271 - mae: 0.5439 - mape: 322.1861 - r2_score_tf: 0.4066 - val_loss: 0.6117 - val_mse: 0.6621 - val_mae: 0.6117 - val_mape: 453.6442 - val_r2_score_tf: 0.3100


In [19]:
finetune_model.load_weights(f'./NN_search/testifinetus_{study_name}.h5')
# finetune_model.save(f'./NN_search/koe', save_format='tf') # TODO tässä ongelmaa vielä, mutta ei väliä. Malli on jo olemassa ja sitä voi käyttää. 


In [20]:


train_images_path = train_df['image_path'].values
valid_images_path = valid_df['image_path'].values

train_dataset = tf.data.Dataset.from_tensor_slices((train_images_path, y_train))
valid_dataset = tf.data.Dataset.from_tensor_slices((valid_images_path, y_valid))

train_dataset = train_dataset.map(process_path_valid, num_parallel_calls=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.map(process_path_valid, num_parallel_calls=tf.data.experimental.AUTOTUNE)

BATCH_SIZE = 10

train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)





In [21]:

## TRAINING DATA TEST
tf.keras.backend.clear_session()
gc.collect()

train_pred = finetune_model.predict(train_dataset, verbose=1)

train_pred = std_scaler.inverse_transform(train_pred)
train_pred = np.power(10, train_pred)

R2_train = r2_score(y_train_original, train_pred)
MSE_train = mean_squared_error(y_train_original, train_pred)
MAE_train = mean_absolute_error(y_train_original, train_pred)
MAPE_train = mean_absolute_percentage_error(y_train_original, train_pred)

print(f'Train scores:\nR2 : {R2_train:.5f}, MSE : {MSE_train:.5f}, MAE : {MAE_train:.5f}, MAPE : {MAPE_train:.5f}')


2024-04-23 11:17:10.873407: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [39335,6]
	 [[{{node Placeholder/_1}}]]
2024-04-23 11:17:10.873663: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [39335,6]
	 [[{{node Placeholder/_1}}]]


3934/3934 [==============================] - 244s 61ms/step
Train scores:
R2 : -inf, MSE : inf, MAE : inf, MAPE : inf


/tmp/ipykernel_282380/3851695042.py:8: RuntimeWarning: overflow encountered in power
  train_pred = np.power(10, train_pred)


In [22]:

## VALIDATION DATA TEST

tf.keras.backend.clear_session()
gc.collect()

valid_pred = finetune_model.predict(valid_dataset, verbose=1)

valid_pred = std_scaler.inverse_transform(valid_pred)
valid_pred = np.power(10, valid_pred)

R2_valid = r2_score(y_valid_original, valid_pred)
MSE_valid = mean_squared_error(y_valid_original, valid_pred)
MAE_valid = mean_absolute_error(y_valid_original, valid_pred)
MAPE_valid = mean_absolute_percentage_error(y_valid_original, valid_pred)

print(f'Valid scores:\nR2 : {R2_valid:.5f}, MSE : {MSE_valid:.5f}, MAE : {MAE_valid:.5f}, MAPE : {MAPE_valid:.5f}')



2024-04-23 11:21:17.740552: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [9833,6]
	 [[{{node Placeholder/_1}}]]
2024-04-23 11:21:17.740795: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [9833,6]
	 [[{{node Placeholder/_1}}]]


984/984 [==============================] - 61s 61ms/step
Valid scores:
R2 : -inf, MSE : inf, MAE : inf, MAPE : inf


/tmp/ipykernel_282380/3183524269.py:9: RuntimeWarning: overflow encountered in power
  valid_pred = np.power(10, valid_pred)


In [29]:
# TEST DATA 


tf.keras.backend.clear_session()
gc.collect()


test_df_copy = test_df.copy()
submission_df = test_df_copy[['id']].copy()

test_dataset = tf.data.Dataset.from_tensor_slices(test_df['image_path'].values)
test_dataset = test_dataset.map(process_path_test, num_parallel_calls=tf.data.experimental.AUTOTUNE)

predictions = finetune_model.predict(test_dataset, verbose=1)
predictions = std_scaler.inverse_transform(predictions)
predictions = np.power(10, predictions)



2024-04-23 13:27:50.405806: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [6545]
	 [[{{node Placeholder/_0}}]]


6545/6545 [==============================] - 262s 39ms/step


/tmp/ipykernel_282380/3318921143.py:19: RuntimeWarning: overflow encountered in power
  predictions = np.power(10, predictions)


In [ ]:

target_columns = ['X4', 'X11', 'X18', 'X50', 'X26', 'X3112']

submission_df[target_columns] = predictions


In [ ]:
submission_df.head()

In [ ]:
print(submission_df.info())

submission_df.to_csv('./data/submission.csv', index=False)